In [83]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from semantic_text_similarity.models import WebBertSimilarity
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy import spatial
import parmap
import os
import swifter
from tqdm import tqdm
import sqlite3

In [84]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [85]:
df = pd.read_csv('./data/bert_partitions.csv')
df = df.drop(['Unnamed: 0', 'Source', 'Program Name', 'Time'], axis=1)

In [86]:
df['Date'] = pd.to_datetime(df['Date'])

In [87]:
df

,Date,Duration,Transcript,partitioned_transcript
0,2014-12-06,20 mins,"#Ah... # Dreaming ofthe days # APPLAUSE Hiya, ...","# Oh, yeah # I see your smiling face # Like I’..."
1,2014-12-07,20 mins,Alex Salmond unveils plans to seek a Westminst...,Alex Salmond unveils plans to seek a Westminst...
2,2014-12-07,10 mins,against the illegal trade in wildlife. He’s ve...,"At the World Bank, he will be making a speech ..."
3,2014-12-05,30 mins,# The way I love you. # Making Christmas speci...,Serious mistakes are made by England’s health ...
4,2014-12-05,25 mins,Two British men are jailed for travelling to S...,Two British men are jailed for travelling to S...
...,...,...,...,...
1238,2014-06-26,30 mins,"Hello, Glastonbury. Hello, Glastonbury! Hello,...",Use the BBC Weather App to stay one step ahead...
1239,2014-06-26,15 mins,A little bit more cloud around but some of us ...,A little bit more cloud around but some of us ...
1240,2014-06-26,25 mins,Yet more shocking re-lations about the extent ...,Yet more shocking re-lations about the extent ...
1241,2014-06-26,30 mins,The most detailed picture yet ofthe The most d...,The most detailed picture yet ofthe The most d...


In [81]:
doc2vec_model = Doc2Vec.load('models/doc2vec/doc2vec_news')

In [82]:
conn = sqlite3.connect('../../data/news.db')

In [38]:
sql = 'select * from articles where year={} and month={} and day between {} and {}'

In [58]:
def infer(transcript):
    return doc2vec_model.infer_vector(preprocess_text(transcript))

In [74]:
from sklearn.metrics.pairwise import cosine_similarity

partitions_preds = []
for index, row in tqdm(list(df.iterrows())):
    partitions = row['partitioned_transcript'].split('\n---------------------\n')
    dt = row['Date']
    statement = sql.format(dt.year, dt.month, dt.day-1, dt.day+1)
    articles = pd.read_sql_query(statement, conn)
    
    sim_max = 0
    topic = -1
    display(articles)
    
    iterations = 0
    for partition in partitions:
        for index2, row2 in tqdm(list(articles.iterrows())):
            transcript_vec = infer(row['Transcript'])
            partition_vec = infer(partition)
            sim = cosine_similarity([transcript_vec, partition_vec])[0][0]

            if sim > sim_max:
                sim_max = sim
                topic = row2['top1_topic']
                iterations=0
            else:
                iterations +=1
            if iterations > 500:
                sim_max = 0
                partitions_preds.append((index, partition, topics_index_to_name_map[topic]))
                break
    
    break
    








  0%|          | 0/1243 [00:00<?, ?it/s]

,source_id,source,day,month,year,program_name,transcript,parliament,top1_topic,top1_acc,top2_topic,top2_acc,top3_topic,top3_acc
0,163795,Belfast Telegraph,5,12,2014,Young chef has the recipe for success with dre...,"Oliver Molloy, a graduate of the NWRC's profes...",None,20,53.66,2,16.68,8,5.17
1,163795,Belfast Telegraph,5,12,2014,30 jobs to go as social security posts getting...,Staff at the Lisahally offices were told the w...,None,20,38.43,15,27.30,4,10.88
2,163795,Belfast Telegraph,5,12,2014,"'Amoosing' evening of fun, frolics and hilarit...",So proclaimed Panto Dame William Caulfield as ...,None,20,66.46,5,4.99,10,4.95
3,163795,Belfast Telegraph,5,12,2014,"Andre Rieu Odyssey Arena, Belfast [...]",The show is an ode to the waltz and is named a...,None,5,26.64,20,18.72,15,8.50
4,163795,Belfast Telegraph,5,12,2014,PICK OF THE WEEK,"Best known for chart hits dating back to 1978,...",None,5,71.22,20,10.72,15,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13340,412338,Wales,7,12,2014,Matthew Myers clinches Cardiff Devils win with...,Man of the match Matthew Myers led Cardiff Dev...,None,20,57.15,5,26.02,16,5.41
13341,412338,Wales,7,12,2014,Matthew Myers clinches Cardiff Devils success ...,Man of the match Matthew Myers led Cardiff Dev...,None,20,62.39,5,20.42,16,6.47
13342,412338,Wales,7,12,2014,Welsh tourist plunges to his death in Australi...,New South Wales Police are yet to confirm the ...,None,20,69.78,5,10.86,15,8.00
13343,412338,Wales,7,12,2014,Tourist plunges to his death in Australia clif...,New South Wales Police are yet to confirm the ...,None,20,43.99,5,39.98,15,5.45










  0%|          | 0/13345 [00:00<?, ?it/s]







  0%|          | 1/13345 [00:00<38:32,  5.77it/s]







  0%|          | 2/13345 [00:00<37:20,  5.95it/s]







  0%|          | 3/13345 [00:00<35:36,  6.25it/s]







  0%|          | 4/13345 [00:00<33:32,  6.63it/s]







  0%|          | 5/13345 [00:00<31:21,  7.09it/s]







  0%|          | 6/13345 [00:00<29:15,  7.60it/s]







  0%|          | 7/13345 [00:00<27:30,  8.08it/s]







  0%|          | 8/13345 [00:01<26:17,  8.46it/s]







  0%|          | 9/13345 [00:01<25:25,  8.74it/s]







  0%|          | 10/13345 [00:01<24:48,  8.96it/s]







  0%|          | 11/13345 [00:01<24:26,  9.09it/s]







  0%|          | 12/13345 [00:01<24:07,  9.21it/s]







  0%|          | 13/13345 [00:01<23:56,  9.28it/s]







  0%|          | 14/13345 [00:01<23:46,  9.34it/s]







  0%|          | 15/13345 [00:01<23:40,  9.38it/s]







  0%|          | 16/13345 [00:01<23:37,  9.40it/s]







  0%|          | 17/1334

  1%|▏         | 198/13345 [00:22<23:04,  9.49it/s]







  1%|▏         | 199/13345 [00:22<23:03,  9.50it/s]







  1%|▏         | 200/13345 [00:22<23:06,  9.48it/s]







  2%|▏         | 201/13345 [00:22<23:06,  9.48it/s]







  2%|▏         | 202/13345 [00:22<23:05,  9.48it/s]







  2%|▏         | 203/13345 [00:22<23:04,  9.49it/s]







  2%|▏         | 204/13345 [00:23<23:06,  9.48it/s]







  2%|▏         | 205/13345 [00:23<23:05,  9.49it/s]







  2%|▏         | 206/13345 [00:23<23:06,  9.47it/s]







  2%|▏         | 207/13345 [00:23<23:05,  9.48it/s]







  2%|▏         | 208/13345 [00:23<23:06,  9.48it/s]







  2%|▏         | 209/13345 [00:23<23:04,  9.49it/s]







  2%|▏         | 210/13345 [00:23<23:04,  9.48it/s]







  2%|▏         | 211/13345 [00:23<23:04,  9.49it/s]







  2%|▏         | 212/13345 [00:23<23:05,  9.48it/s]







  2%|▏         | 213/13345 [00:24<23:04,  9.48it/s]







  2%|▏         | 214/13345 [00:24<23:07,  9.46it/s]






  3%|▎         | 394/13345 [00:44<22:44,  9.49it/s]







  3%|▎         | 395/13345 [00:44<22:44,  9.49it/s]







  3%|▎         | 396/13345 [00:44<22:41,  9.51it/s]







  3%|▎         | 397/13345 [00:44<22:42,  9.50it/s]







  3%|▎         | 398/13345 [00:44<22:42,  9.50it/s]







  3%|▎         | 399/13345 [00:44<22:44,  9.49it/s]







  3%|▎         | 400/13345 [00:44<22:46,  9.48it/s]







  3%|▎         | 401/13345 [00:44<22:44,  9.48it/s]







  3%|▎         | 402/13345 [00:44<22:42,  9.50it/s]







  3%|▎         | 403/13345 [00:44<22:42,  9.50it/s]







  3%|▎         | 404/13345 [00:45<22:42,  9.50it/s]







  3%|▎         | 405/13345 [00:45<22:41,  9.51it/s]







  3%|▎         | 406/13345 [00:45<22:40,  9.51it/s]







  3%|▎         | 407/13345 [00:45<22:41,  9.50it/s]







  3%|▎         | 408/13345 [00:45<22:41,  9.50it/s]







  3%|▎         | 409/13345 [00:45<22:42,  9.49it/s]







  3%|▎         | 410/13345 [00:45<22:41,  9.50it/s]






...

  3%|▎         | 385/13345 [00:41<23:10,  9.32it/s]

















  3%|▎         | 386/13345 [00:41<23:06,  9.35it/s]

















  3%|▎         | 387/13345 [00:41<23:04,  9.36it/s]

















  3%|▎         | 388/13345 [00:41<23:03,  9.36it/s]

















  3%|▎         | 389/13345 [00:41<23:03,  9.37it/s]

















  3%|▎         | 390/13345 [00:41<23:01,  9.37it/s]

















  3%|▎         | 391/13345 [00:41<23:01,  9.37it/s]

















  3%|▎         | 392/13345 [00:41<22:59,  9.39it/s]

















  3%|▎         | 393/13345 [00:42<23:00,  9.38it/s]

















  3%|▎         | 394/13345 [00:42<23:02,  9.37it/s]

















  3%|▎         | 395/13345 [00:42<23:03,  9.36it/s]

















  3%|▎         | 396/13345 [00:42<23:01,  9.37it/s]

















  3%|▎         | 397/13345 [00:42<23:03,  9.36it/s]

















  3%|▎         | 398/13345 [00:42<23:02,  9.37it/s]

















  3%|▎         | 399/13345 [00:42<

  4%|▍         | 517/13345 [00:55<22:53,  9.34it/s]

















  4%|▍         | 518/13345 [00:55<22:53,  9.34it/s]

















  4%|▍         | 519/13345 [00:55<22:51,  9.35it/s]

















  4%|▍         | 520/13345 [00:55<22:51,  9.35it/s]

















  4%|▍         | 521/13345 [00:55<22:50,  9.36it/s]

















  4%|▍         | 522/13345 [00:55<22:48,  9.37it/s]

















  4%|▍         | 523/13345 [00:55<22:49,  9.37it/s]

















  4%|▍         | 524/13345 [00:56<22:48,  9.37it/s]

















  4%|▍         | 525/13345 [00:56<22:48,  9.37it/s]

















  4%|▍         | 526/13345 [00:56<22:48,  9.37it/s]

















  4%|▍         | 527/13345 [00:56<22:47,  9.37it/s]

















  4%|▍         | 528/13345 [00:56<22:47,  9.37it/s]

















  4%|▍         | 529/13345 [00:56<22:50,  9.35it/s]

















  4%|▍         | 530/13345 [00:56<22:48,  9.37it/s]

















  4%|▍         | 531/13345 [00:56<

 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidde

 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidde

 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidde

In [75]:
partitions_preds

[(0,
  '# Oh, yeah # I see your smiling face # Like I’ve never seen before... # Isn’t it wonderful? # I wish you a merry Christmas, baby... # I love you so much. # And lots of happiness in the coming year... # This Christmas on BBC One.',
  'Parliament, government and politics'),
 (0,
  'Two hostages are killed in Yemen during a failed rescue attempt by the United States. They were trying to free British-born Luke Somers, and a South African captive.',
  'Health services and medicine'),
 (0,
  'Our hearts are full of sorrow tonight, our prayers and thoughts go out to the Sommers family.',
  'Business, industry and consumers'),
 (0,
  'Ferocious winds and torrential rain in the eastern Philippines, as Typhoon Hagupit hits land And footballers across the country commemorate the centenary of the First World War- and the Christmas Truce of 1914.',
  'Health services and medicine'),
 (0,
  'President Obama has condemned as "barbaric" the killing of two hostages in Yemen by al-Qaeda militant